### Sending in redirect to our search engine

In [ ]:
import requests
import pprint
import json

pp = pprint.PrettyPrinter(indent=4)

In [ ]:
request_headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "no-cache",
    "Connection": "keep-alive",
    "Cookie": "JSESSIONID=2F6E7BF66D9BAE0846E5313DBFDEA42B.tomcat-13409",
    "Host": "lsu.ent.sirsi.net",
    "Pragma": "no-cache",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
    }

In [ ]:
data = {
    'qu': 'asdf',
    'qf': 'FORMAT	Format	BOOK	Books',
#     'rt': 'false|||AUTHOR|||Author',
#     'qf': 'LIBRARY	Library	21:MIDL-MAIN	Middleton Library (Main Collection)'
#     'libraryDropDown': '21:MIDL-MAIN',
    
}

In [ ]:
response = requests.post("https://lsu.ent.sirsi.net/client/en_US/lsu/search/results",
#                        headers=request_headers,
                         data=data
                         )
print(response.status_code)

Lessons:

- post is required here for sending headers & data.
- The tab version seems necessary.
- Headers are not necessary; sending in data is sufficient
- qu is the code for the search term
- qf is the code for the search hard limiters
- rt is the code for the search soft limiters


In [ ]:
with open('trash.html', 'w') as f:
    f.write(response.content.decode('utf-8'))

In [ ]:
data = {
    'qu': 'asdf',
#     'qf': 'FORMAT	Format	BOOK	Books',
#     'rt': 'false|||AUTHOR|||Author',
    'qf': 'LIBRARY	Library	21:MIDL-MAIN	Middleton Library (Main Collection)'
    
}

response = requests.post("https://lsu.ent.sirsi.net/client/en_US/lsu/search/results?",
#                        headers=request_headers,
                         data=data
                         )
print(response.status_code)
print(response.content)

In [ ]:
def de_urlize(text):
    text = text.replace('%09', '	').replace('%3A', ':').replace('%28', '(').replace('%29', ')').replace('+', ' ').replace('\t', '	').replace('%7C', '|')
    return text

In [ ]:
a = de_urlize('https://lsu.ent.sirsi.net/client/en_US/lsu/search/results?qu=asdf&qf=LIBRARY%09Library%0921%3AMIDL-MAIN%09Middleton+Library+%28Main+Collection%29')
b = 'https://lsu.ent.sirsi.net/client/en_US/lsu/search/results?qu=asdf&qf=LIBRARY	Library	21:MIDL-MAIN	Middleton Library (Main Collection)'

In [ ]:
assert a == b

In [ ]:
de_urlize('https://lsu.ent.sirsi.net/client/en_US/lsu/search/results?qu=asdf&rt=&qf=LIBRARY%09Library%0921%3AVETMED%09Veterinary+Medicine+Library&qf=&qf=&qf=&qf=&av=&advancedSearchButton=Search')


### Sending in a broken redirect (Doesn't work.)

####    Example embedded search

In [ ]:
data = {
    'ln': 'en_US',
    'rt': 'true|||AUTHOR|||Author',
    'q': 'Schoenberg',
#     'rw': '0',
#     'lm': 'FORMAT',
#     'qf': 'FORMAT\tFormat\tMUSICSNDREC\tMusic+Sound+Recording',
#     'id': 'q',
}

In [ ]:
response = requests.post("https://lsu.ent.sirsi.net/client/embedded.search/lsu",
             headers=request_headers,
             data=data
            )
print(response.status_code)

In [ ]:
with open('trash.html', 'w') as f:
    f.write(response.content.decode('utf-8'))

Lessons:

 - This url is recommended at: http://www.mediatheque.mairie-champigny94.fr/client/help/admin-help_en_US/Content/admin-help/2-Getting%20started/Inserting%20the%20%20Search%20bo.htm#vse-getstart_1270969419_1269116
 
 - It seems fickle & I cant find reliable data to send in.
 

In [ ]:
print(dir(response.request))
print(response.request)


### Hitting Symphony web API & getting info

In [ ]:
def stringify_params(params):
    params_as_str = ''
    for count, (k, v) in enumerate(params.items()):
        str_a = f'{k}={v}'
        if count == 0:
            params_as_str += str_a
        else:
            params_as_str += f'&{str_a}'
    return params_as_str

In [ ]:
webservices_base_url = 'https://lalu.sirsi.net/lalu_ilsws'
service = 'standard'
function = 'lookupTitleInfo'
params = {
    'clientID': 'DS_CLIENT',
    'json': 'true',
    'prettyprint': 'true',
    'titleID': '2644971',
    'includeAvailabilityInfo': 'true',
    'includeItemInfo': 'true',
    'includeOPACInfo': 'true',
    'includeCatalogingInfo': 'true',
    
}

In [ ]:
# Example Book
params['titleID'] = '1386605'

In [ ]:
# Example Periodical
params['titleID'] = '316188'

In [ ]:
composed_url = f'{webservices_base_url}/rest/{service}/{function}?{stringify_params(params)}'
print(composed_url)

In [ ]:
response = requests.get(composed_url)

In [ ]:
print(response.text)

### Following MARC Holdings Table Widget

In [ ]:
baseWsURL = 'https://lalu.sirsi.net/lalu_ilsws/'
clientID = 'DS_CLIENT'
catKey = '2805471'

locationWsURL = """{baseWsURL}rest/admin/lookupLocationPolicyList?clientID={clientID}&json=true"""
libraryWsURL = """{baseWsURL}rest/admin/lookupLibraryPolicyList?clientID={clientID}&json=true"""
materialTypeWsURL = """{baseWsURL}rest/admin/lookupItemTypePolicyList?clientID={clientID}&json=true"""

location_url = locationWsURL.format(baseWsURL=baseWsURL,
                 clientID=clientID,
                 catKey=catKey)
library_url = libraryWsURL.format(baseWsURL=baseWsURL,
                 clientID=clientID,
                 catKey=catKey)
material_url = materialTypeWsURL.format(baseWsURL=baseWsURL,
                 clientID=clientID,
                 catKey=catKey)

In [ ]:
location_policy = requests.post(location_url).text
library_policy = requests.post(library_url).text
material_policy = requests.post(material_url).text

In [ ]:
def simplify_json(response_text):
    dict_a = dict()
    for policy_name, policy_list in json.loads(response_text).items():
        for group in policy_list:
            dict_a[group.get('policyID')] = group.get('policyDescription')
    return dict_a

In [ ]:
location_dict = simplify_json(location_policy)
library_dict = simplify_json(library_policy)
material_dict = simplify_json(material_policy)

In [ ]:
baseWsURL = 'https://lalu.sirsi.net/lalu_ilsws/'
clientID = 'DS_CLIENT'
# catKey = '2805471' # Gay Butler Plater
catKey = '316195' # Korea Journal

options = {"includeAvailabilityInfo": "true",
    "includeCallNumberSummary": "true",
    "includeItemInfo": "true",
    "includeCatalogingInfo": "true",
    "includeOrderInfo": "true",
    "includeOPACInfo": "true",
    "includeBoundTogether": "true",
    "includeMarcHoldings": "true",
    "marcEntryFilter": "ALL",
    "json": "true"}
detail_template = """{baseWsURL}rest/standard/lookupTitleInfo?clientID={clientID}&titleID={catKey}"""
url_options = "".join('&{}={}'.format(k, v) for k, v in options.items())
detail_template += url_options
detail_url = detail_template.format(baseWsURL=baseWsURL,
                 clientID=clientID,
                 catKey=catKey)
print(detail_url)

In [ ]:
detail_json = requests.post(detail_url).text

In [ ]:
response = requests.post(detail_url)


In [ ]:
response.text

In [ ]:
for k, items_list in json.loads(detail_json).items():
    for item_dict in items_list:
        for k, v in item_dict.items():
            if k == 'MarcHoldingsInfo':
                for marc_holding in v:
                    LibraryID = marc_holding.get('holdingLibraryID')
                    MarcEntryInfo = marc_holding.get('MarcEntryInfo')
                    for booger in MarcEntryInfo:
                        label = booger.get('label')
                        alternateLabel = booger.get('alternateLabel')
                        entryID = booger.get('entryID')
                        indicators = booger.get('indicators')
                        text = booger.get('text')
                        unformattedText = booger.get('unformattedText')
                        entryTypeCodes = booger.get('entryTypeCodes')
                        url = booger.get('url')
                        relatedLookupText = booger.get('relatedLookupText')
                        relatedLookupIndexID = booger.get('relatedLookupIndexID')
                        relatedLookupHeadingTypeID = booger.get('relatedLookupHeadingTypeID')
                        if entryID == '852':
                            print(LibraryID)
                            for o,p in booger.items():
                                print(o, p)
                            
                        

In [ ]:
library_dict.get('SPEC_COLL')

#### Looking for redirect links

In [ ]:
from bs4 import BeautifulSoup

starting_url = "https://www.lib.lsu.edu/"
response = requests.get(starting_url, verify=False)

In [ ]:
dir(response)

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
links_with_text = [a['href'] for a in soup.find_all('a', href=True) if a.text and a['href'].startswith('http')]

In [ ]:

links_with_text

In [ ]:
for link in links_with_text:
    try:
        response = requests.get(link)
    except requests.exceptions.SSLError:
        print(link)
        response = requests.get(link, verify=False)
    print(response.status_code)